In [4]:
from pathlib import Path
from matplotlib.style import use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn

path = Path.cwd().parent.parent/'Data/processed/targetCows/'
usecols = ['MilkingEventDateTime', 'FarmName_Pseudo', 'TrafficDeviceName', 'MilkProduction', 'timeDelta_Seconds', 'LactationNumber', 'DaysInMilk', 'BreedName', 'Age']
lacNum = 1
lacNum_next = 2
threshold_rate = 0.45

# labeling cow with good/bad learner(0/1)
def labeling_data(lacNum, lacNum_next, threshold_rate, cow_total):
    global learner
    timeCost1 = cow_total.loc[cow_total.LactationNumber == lacNum].timeDelta_Seconds.mean()
    timeCost2 = cow_total.loc[cow_total.LactationNumber == lacNum_next].timeDelta_Seconds.mean()
    timediff = timeCost1-timeCost2
    #print('result: '+str(timeCost1)+'  '+str(timeCost2)+'  '+str(timediff/timeCost1))
    if timediff > 0:
        if timediff/timeCost1 >= threshold_rate:
            learner = 1 # good learner
        else:
            learner = 0 # bad learner
    cow_total['label'] = learner
    return cow_total

In [5]:
# integrate all the cows data into one dataset
filelist = list(Path(path).glob('*.*'))
for i, file in enumerate(filelist):
    single_cow = pd.read_csv(file, encoding='utf-8', usecols=usecols, index_col='MilkingEventDateTime')
    single_cow = labeling_data(lacNum, lacNum_next, threshold_rate, single_cow)
    if i == 0:
        cow_total = single_cow
    cow_total = pd.concat([cow_total, single_cow])
cow_total.sort_values(by=['MilkingEventDateTime'], inplace=True)

In [6]:
#df_static_features = cow_total[['FarmName_Pseudo']]
categorical_cols = ['TrafficDeviceName', 'LactationNumber', 'BreedName']
numerical_cols = ['Age', 'MilkProduction', 'timeDelta_Seconds', 'DaysInMilk']
output_col = ['label']

for col in categorical_cols:
    cow_total[col] = cow_total[col].astype('category')


In [11]:
device = cow_total.TrafficDeviceName.cat.codes.values
lac = cow_total.LactationNumber.cat.codes.values
breed = cow_total.BreedName.cat.codes.values
categorical_features = np.stack([device, lac, breed], 1)


array([[1, 0, 0],
       [1, 0, 1],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1]], dtype=int8)

In [59]:
cow_total

,FarmName_Pseudo,TrafficDeviceName,MilkProduction,timeDelta_Seconds,LactationNumber,DaysInMilk,BreedName,Age,label
MilkingEventDateTime,,,,,,,,,
2020-08-17 05:49:08,a624fb9a,vms2,7.10,621.0,1.0,11.0,1,4.10,0
2020-08-17 08:38:47,a624fb9a,vms2,8.49,4432.0,1.0,10.0,2,4.17,0
2020-08-17 13:03:21,a624fb9a,vms2,7.29,1075.0,1.0,11.0,1,4.10,0
2020-08-17 14:19:46,a624fb9a,vms1,15.90,2186.0,1.0,109.0,1,4.60,0
2020-08-17 19:51:27,a624fb9a,vms1,7.04,1209.0,1.0,11.0,1,4.10,0
...,...,...,...,...,...,...,...,...,...
2022-08-23 20:35:54,a624fb9a,vms2,10.90,11.0,3.0,60.0,1,4.60,0
2022-08-23 21:01:55,a624fb9a,vms1,8.72,12.0,3.0,17.0,1,3.98,1
2022-08-23 22:31:03,a624fb9a,vms2,10.80,16.0,3.0,15.0,1,4.06,1


In [ ]:
trainingSet, testSet = train_test_split(df, test_size=0.2)